# Full pipeline:
0. **Universal pre-processing**
- Include core question variables + age + race + BMI
- Label categorical columns as pandas.CategoricalDtype
- Impute 0 for applicable variables with missing values

---


1. **Handling NaN and don't know / refused**

Based on our available data, our level of domain knowledge, and available methods, modeling the mechanism of missingness for our dataset, if our data is MNAR, is an intractable problem. It is also reasonable to believe that our data is MNAR because for any variable, for example income level, it is reasonable to believe that the mechanism of missingness depends at least to some extent on that variable. Additionally, because of the nature of our data originating in a survey, we notice that sequentially the number of missing answers increases, likely due to respondents ending the survey early.

We also having missing data in the form of "Don't know" and "Refused" answers. While these two answers do differ in their meaning, for the purposes of our project we treat them the same for numerical variables due to it being intractable to impute these with different methods that reflect the semantics of the answer. For categorical variables, we retain "Don't know" and "Refused" as valid categories, under the motivation that they may hold information that would be lost when imputing or dropping.

Based on the above, in order to experiment with our data, we initially process our data in three different ways, described below, making various strong assumptions regarding our data being MAR and MCAR.

\

**(1) Remove rows with NaN, except the ones we impute as 0. Remove rows with don't know/refused in numerical data. Keep don't know/refused for categorical data, since they can still be used as categories. Do this for both train and test splits.**

Result: Assume MCAR, so we drop all missing values, and values of "don't know" or "refused" for numerical features. Because we are dropping rows in this version of the dataset, results from training and testing will not be directly comparable to (2) and (3), which do not drop rows. Classifiers that can be used in this case: decision tree, random forest, XGBoost.

\

**(2) Keep NaN rows for estimators that can handle NaN. Change don't know / refused to NaN for numerical variables. Keep don't know / refused as categories for categorical data. Categorical Variables may still have NaN values. Do this for both train and test splits.**

Result: Assume that missing values themselves hold some information without imputation, and we might be able to leverage this. Also assume that don't know / refused hold information as categories.

\

**(3) Keep rows with NaN. Change don't know / refused to NaN for numerical variables, and keep as a category for categorical. Use IterativeImputer with estimator=HistGradientBoostingRegressor (This estimator natively handles categorical variables - need to cite scikit-learn documentation about this) to impute the missing numerical values and any missing categorical values that are not don't know / refused. Do this for both train and test splits. Fit the imputer on train and transform test.**

Result: Assume that data is MAR, so that we can impute based on the available non-missing values. Assume that don't know / refused holds information as a category, so we keep those and do not impute.


---


2. **Dimension Reduction**

For each of the above, use FAMD from the Prince library (https://maxhalford.github.io/prince/famd/) for dimension reduction. This supports mixed data. Fit on train splits and transform test splits.

Result: Parrallel dataframes for (1) and (3), but with only principal components. Because (2) has non-imputed NaN values for missing data, the Prince implementation of FAMD is unable to handle it. Furthermore, if an implementation of FAMD that can handle non-imputed missing values was used, it would do so by imputing, essentially recreating the initial steps to construct (2), just with possibly a different imputation method. For this case, we will consider the dataframe for (3) post-FAMD to be applicable to both the initial (2) and initial (3) dataframes.

Cite:
@software{Halford_Prince,
    author = {Halford, Max},
    license = {MIT},
    title = {{Prince}},
    url = {https://github.com/MaxHalford/prince}
}


---
3. **Handling unbalanced data**

To handle unbalanced data, we utilize the class_weight='balanced' parameter native in scikit-learn.

"The “balanced” mode uses the values of y to automatically adjust weights inversely proportional to class frequencies in the input data as n_samples / (n_classes * np.bincount(y))" (Cite documentation for each of the methods we use).

For xgboost, we use the parameter scale_pos_weight, which adjusts weights for the positive and negative classes (https://xgboost.readthedocs.io/en/stable/parameter.html)

We further considered using SMOTE-NC to construct synthetic samples to artificially inflate our minority class (https://imbalanced-learn.org/stable/references/generated/imblearn.over_sampling.SMOTENC.html). However, SMOTE-NC uses a nearest neighbors method to construct synthetic samples, and this may lead to potentially logically-inconsistent synthetic samples due to correlation between some of the variables in the complete dataset (e.g., a person who says they do not engage in exercise logically engages in 0 minutes of , but a synthetic sample may contradict this).

---
4. Train/test all models
- logistic regression
- decision tree
- random forest
- AdaBoost
- HistGradientBoostingClassifier
- LightGBM
- XGBoost

Report mean accuracy, precision, recall, and F1 score for training and evaluating using k-fold cross validation


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive/')

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# Read the original XPT data
xpt_path = 'LLCP2023.XPT'
df_xpt = pd.read_sas(xpt_path, format='xport')
df_xpt.head()

,_STATE,FMONTH,IDATE,IMONTH,IDAY,IYEAR,DISPCODE,SEQNO,_PSU,CTELENM1,...,DROCDY4_,_RFBING6,_DRNKWK2,_RFDRHV8,_FLSHOT7,_PNEUMO3,_AIDTST4,_RFSEAT2,_RFSEAT3,_DRNKDRV
0,1.0,1.0,b'03012023',b'03',b'01',b'2023',1100.0,b'2023000001',2.023000e+09,1.0,...,5.397605e-79,1.0,5.397605e-79,1.0,2.0,2.0,2.0,1.0,1.0,9.0
1,1.0,1.0,b'01062023',b'01',b'06',b'2023',1100.0,b'2023000002',2.023000e+09,1.0,...,5.397605e-79,1.0,5.397605e-79,1.0,1.0,1.0,2.0,1.0,1.0,9.0
2,1.0,1.0,b'03082023',b'03',b'08',b'2023',1100.0,b'2023000003',2.023000e+09,1.0,...,5.397605e-79,1.0,5.397605e-79,1.0,1.0,1.0,2.0,1.0,1.0,9.0
3,1.0,1.0,b'03062023',b'03',b'06',b'2023',1100.0,b'2023000004',2.023000e+09,1.0,...,5.397605e-79,1.0,5.397605e-79,1.0,1.0,1.0,1.0,1.0,1.0,9.0
4,1.0,1.0,b'01062023',b'01',b'06',b'2023',1100.0,b'2023000005',2.023000e+09,1.0,...,7.000000e+00,1.0,4.700000e+01,1.0,2.0,1.0,2.0,1.0,1.0,2.0


In [ ]:
# Convert XPT to CSV and save a copy
csv_path = 'full_data.csv'
df_xpt.to_csv(csv_path, index=False)

In [ ]:
# Our target variable
# Do you currently have symptoms lasting 3 months or longer that you did not have prior to having coronavirus or COVID-19?
# Select for yes/no
df_covid_full = df_xpt[df_xpt['COVIDSM1'].isin([1.0, 2.0])]
df_covid_full['COVIDSM1'].value_counts(dropna=False)

COVIDSM1
2.0    170893
1.0     27074
Name: count, dtype: int64

In [ ]:
# use core question variables and _AGE80 (imputed age), _IMPRACE (race, or imputed based on most common race in state), and _BMI5 (BMI calculated from height and weight)
core_start_index = df_covid_full.columns.get_loc('SEXVAR')
core_end_index = df_covid_full.columns.get_loc('COVIDSM1')
core_columns = df_covid_full.columns[core_start_index:core_end_index + 1].tolist()
columns_to_use = ['_AGE80', '_IMPRACE', '_BMI5'] + core_columns
df = df_covid_full[columns_to_use].copy()
print(df['COVIDSM1'].value_counts(dropna=False))

"""
Additional variables that we drop.
Some of these are asked for surveying purposes (phone questions),
some are calendar dates asked as follow-up questions,
and the last two are about having had COVID and a follow-up to long COVID
"""
df.drop('DIABAGE4', axis=1, inplace=True) # How old were you when you were first told you had diabetes? (Missing for non-diabetics)
df.drop('NUMHHOL4', axis=1, inplace=True) # Not including cell phones or numbers used for computers, fax machines or security systems, do you have more than one landline telephone number in your household?
df.drop('NUMPHON4', axis=1, inplace=True) # How many of these landline telephone numbers are residential numbers?
df.drop('CPDEMO1C', axis=1, inplace=True) # How many cell phones do you have for your personal use?
df.drop('FLSHTMY3', axis=1, inplace=True) # During what month and year did you receive your most recent flu vaccine that was sprayed in your nose or flu shot injected into your arm?
df.drop('HIVTSTD3', axis=1, inplace=True) # Not including blood donations, in what month and year was your last H.I.V. test?
df.drop('COVIDPO1', axis=1, inplace=True) # Have you ever tested positive for COVID-19?

df.head() # dataframe with all potential variables to use

COVIDSM1
2.0    170893
1.0     27074
Name: count, dtype: int64


,_AGE80,_IMPRACE,_BMI5,SEXVAR,GENHLTH,PHYSHLTH,MENTHLTH,POORHLTH,PRIMINS1,PERSDOC3,...,AVEDRNK3,DRNK3GE5,MAXDRNKS,FLUSHOT7,PNEUVAC4,SHINGLE2,HIVTST7,SEATBELT,DRNKDRI2,COVIDSM1
5,62.0,2.0,3018.0,2.0,3.0,2.0,3.0,88.0,7.0,2.0,...,2.0,88.0,2.0,2.0,2.0,2.0,1.0,1.0,88.0,2.0
7,78.0,2.0,2727.0,2.0,4.0,1.0,88.0,1.0,3.0,1.0,...,NaN,NaN,NaN,1.0,1.0,1.0,2.0,1.0,NaN,2.0
8,80.0,2.0,3347.0,2.0,3.0,5.0,88.0,88.0,3.0,2.0,...,NaN,NaN,NaN,1.0,1.0,1.0,2.0,1.0,NaN,2.0
10,59.0,1.0,4184.0,1.0,3.0,88.0,88.0,NaN,1.0,1.0,...,NaN,NaN,NaN,2.0,2.0,2.0,2.0,3.0,NaN,1.0
11,78.0,1.0,2413.0,2.0,2.0,2.0,88.0,88.0,3.0,1.0,...,NaN,NaN,NaN,1.0,2.0,2.0,2.0,1.0,NaN,2.0


Some values are encoded in ways that are not useable in their raw form for our purposes. Some examples: 101 = "1 time per week"
and 201 = "1 time per month", or 130 being "1 hour and 30 minutes".
The original data includes some additional calculated variables that do this
processing, but they automatically convert to a blank/missing value in some
cases when it may be reasonable to impute the value as 0.
E.g., if someone said they do not engage in physical exercise, the survey
encodes the amount of time they spend on physical exercise as missing, when it
is reasonable to impute 0.

There are also some follow-up questions where missing values can be imputed as 0. For example, If someones answers "None" to one or both of "For how many days during the past 30 days was your physical health not good?" and "For how many days during the past 30 days was your mental health not good?", then they're value for the question "During the past 30 days, for about how many days did poor physical or mental health keep you from doing your usual activities, such as self-care, work, or recreation?" is recorded as blank (they are not asked this question). For variables corresponding to such questions, we overwrite applicable missing values with the imputed value 0.

We process all answers of "Don't know" and "Refused" for numerical data to be missing values.

We add a "Missing" category (encoded as -1) to a few features that are follow-up questions where the follow-up is not asked if the original answer is something like "Never", "Don't know", or "Refused". We motivate this by identifying "Missing" to be distinct from a "No" answer to the follow up. For example, if someone has never had their cholesterol checked, then they presumably

We handle these processing steps now.

In [ ]:
# helper functions
def convert_height(value):
    # Convert height to meters.
    # Originally in form "xyz" for "x feet and yz inches"
    # or "9xyz" for "xyz centimeters" (leading 9 indicates metric)

    if pd.isna(value):
        return np.nan

    val = int(value)

    if 200 <= val <= 211:
        inches = (val - 200) + 24
    elif 300 <= val <= 311:
        inches = (val - 300) + 36
    elif 400 <= val <= 411:
        inches = (val - 400) + 48
    elif 500 <= val <= 511:
        inches = (val - 500) + 60
    elif 600 <= val <= 611:
        inches = (val - 600) + 72
    elif 700 <= val <= 711:
        inches = (val - 700) + 84
    elif 9000 <= val < 9999:
        cm = val - 9000
        return cm * 0.01
    else: # just in case there's some erroneous value
        return np.nan

    meters = inches * 0.0254
    return meters

def convert_weight(value):
    # Convert weight to kg
    # If between 50 - 776, weight is in pounds, convert to kg
    # If 9023 - 9352: subtract 9000 to get kg
    # 7777 = Don't know
    # 9999 = Refused
    # These ranges are defined in the 2023 BRFSS codebook

    if pd.isna(value):
        return np.nan

    val = int(value)

    if 50 <= val <= 776:
        return val * 0.4535924
    elif 9000 <= val <= 9352:
        return val - 9000
    else:
        return np.nan

def convert_frequency_field(value):
    # For fields like EXEROFT1: 101 - 199 means times per week = (value - 100)
    # 201 - 299 means times per month = (value - 200)

    if pd.isna(value):
        return np.nan

    val = int(value)

    if 101 <= val <= 199:
        # times per week
        return val - 100
    elif 201 <= val <= 299:
        # times per month
        return (((val - 200)/30)*7)
    elif val in [777, 999]: # catch these if not already replaced
        return np.nan
    else:
        return val

def convert_hmm_field(value):
    # For fields like EXERHMM1:
    # 777 represents "Don't know"
    # 999 represents "Refused"
    # All other values HMM represent H hours and MM minutes

    if pd.isna(value):
        return np.nan
    val = int(value)
    if 1 <= val <= 759:
        h = val // 100
        m = val % 100
        return h * 60 + m
    elif 800 <= val <= 959:
        h = val // 100
        m = val % 100
        return h * 60 + m
    elif val in [777, 999]:
        return np.nan
    else:
        return val

In [ ]:
# if a person has no days when their mental or physical health was bad,
# they have no days when it kept them from doing their usual activities
df['POORHLTH'] = df['POORHLTH'].replace({np.nan:0})
for col in ['PHYSHLTH','MENTHLTH','POORHLTH']:
    df[col] = df[col].replace({88:0})
    df[col] = df[col].apply(lambda x: x if (0 <= x <= 30) else np.nan)

# if a person doesn't exercise, they spend 0 minutes per month on their primary exercise
df.loc[(df['EXERANY2'] == 2) & (df['EXERHMM1'].isna()), 'EXERHMM1'] = 0
df['EXERHMM1'] = df['EXERHMM1'].apply(convert_hmm_field)

# if a person doesn't exercise, or doesn't have a secondary exercise,
# they spend 0 minutes per month on their secondary exercise
df.loc[(df['EXERANY2'] == 2) & (df['EXERHMM2'].isna()), 'EXERHMM2'] = 0 # no exercise at all
df.loc[(df['EXRACT22'] == 88) & (df['EXERHMM2'].isna()), 'EXERHMM2'] = 0 # no secondary exercise

# missing category for category of primary/secondary exercise
df['EXRACT12'] = df['EXRACT12'].fillna(-1)
df['EXRACT22'] = df['EXRACT22'].fillna(-1)

# frequency of exercises
df.loc[(df['EXERANY2'] == 2) & (df['EXEROFT1'].isna()), 'EXEROFT1'] = 0
df.loc[(df['EXERANY2'] == 2) & (df['EXEROFT2'].isna()), 'EXEROFT2'] = 0 # no exercise
df.loc[(df['EXRACT22'].isna()) & (df['EXEROFT2'].isna()), 'EXEROFT2'] = 0 # not asked due to previous answers
df.loc[(df['EXRACT22'] == 88) & (df['EXEROFT2'].isna()), 'EXEROFT2'] = 0 # no second exercise

# frequency of strength exercises
df['STRENGTH'] = df['STRENGTH'].replace({888:0, 777:np.nan, 999:np.nan})
df['STRENGTH'] = df['STRENGTH'].apply(convert_frequency_field)

# if you've never been told you have high blood pressure, or you don't know,
# we assume you don't take a prescription medicine for high blood pressure
# if you refused to answer before, then you're still coded as blank
# don't know and refused are kept as categories
df.loc[(df['BPHIGH6'].isin([3, 7])) & (df['BPMEDS1'].isna()), 'BPMEDS1'] = 2
df['BPMEDS1'] = df['BPMEDS1'].fillna(-1)

# if you've never had your cholesterol checked (or don't know/refused),
# then you've presumably never been told by a doctor that you have high cholesterol
# but this is distinct from having had it checked and being told it's not high
df['TOLDHI3'] = df['TOLDHI3'].fillna(-1)

# same for taking cholesterol medicine
df['CHOLMED3'] = df['CHOLMED3'].fillna(-1)

# same logic for no previous asthma & no current asthma
df['ASTHNOW'] = df['ASTHNOW'].fillna(-1)

# number of children in household
df['CHILDREN'] = df['CHILDREN'].replace({88:0, 99:np.nan})

# "Are you now pregnant?" is not asked to women above 49 or men
df['PREGNANT'] = df['PREGNANT'].fillna(-1)

# standardize weight to kg
df['WEIGHT2'] = df['WEIGHT2'].apply(convert_weight)

# standardize height to meters
df['HEIGHT3'] = df['HEIGHT3'].apply(convert_height)

# number of falls in past year, not asked to people 18-44
# reinterprets variable to something like "number of elderly falls in past year"
df['FALL12MN'] = df['FALL12MN'].replace({88:0, np.nan:0})
df['FALL12MN'] = df['FALL12MN'].replace({77:np.nan, 99:np.nan})

# falls that caused injury
df['FALLINJ5'] = df['FALLINJ5'].replace({88:0, np.nan:0})
df['FALLINJ5'] = df['FALLINJ5'].replace({77:np.nan, 99:np.nan})

# not asked if you haven't smoked at least 100 cigarrettes
df['SMOKDAY2'] = df['SMOKDAY2'].fillna(-1)

# frequency of drinking
df['ALCDAY4'] = df['ALCDAY4'].replace({888:0, 777:np.nan, 999:np.nan})
df['ALCDAY4'] = df['ALCDAY4'].apply(convert_frequency_field)

# 0 for nondrinkers
# (average number of drinks, frequency of binge drinking, largest number of drinks on one occasion in past month)
mask_alc = (df['ALCDAY4']==0)
for c in ['AVEDRNK3','DRNK3GE5','MAXDRNKS']:
    df.loc[mask_alc & df[c].isna(), c] = 0
df['AVEDRNK3'] = df['AVEDRNK3'].replace({77:np.nan, 99:np.nan, 88:0})  # some people say they drink but on average drink 0 drinks when they drink
df['DRNK3GE5'] = df['DRNK3GE5'].replace({77:np.nan, 99:np.nan, 88:0})
df['MAXDRNKS'] = df['MAXDRNKS'].replace({77:np.nan, 99:np.nan, 88:np.nan}) # 88 in this column is coded as "Invalid response"

# shingles vaccine not asked to people under 50
df['SHINGLE2'] = df['SHINGLE2'].fillna(-1)

# if you don't drive, or don't drink, you don't drink and drive
df['DRNKDRI2'] = df['DRNKDRI2'].replace({np.nan:0, 88:0})
df['DRNKDRI2'] = df['DRNKDRI2'].replace({77:np.nan, 99:np.nan})

# encode target variable COVIDSM1: 1.0 -> 1 for yes, 2.0 -> 0 for no
label_map = {1.0: 1, 2.0: 0}
df['COVIDSM1'] = df['COVIDSM1'].map(label_map)


In [ ]:
df.head()

,_AGE80,_IMPRACE,_BMI5,SEXVAR,GENHLTH,PHYSHLTH,MENTHLTH,POORHLTH,PRIMINS1,PERSDOC3,...,AVEDRNK3,DRNK3GE5,MAXDRNKS,FLUSHOT7,PNEUVAC4,SHINGLE2,HIVTST7,SEATBELT,DRNKDRI2,COVIDSM1
5,62.0,2.0,3018.0,2.0,3.0,2.0,3.0,0.0,7.0,2.0,...,2.0,0.0,2.0,2.0,2.0,2.0,1.0,1.0,0.0,0
7,78.0,2.0,2727.0,2.0,4.0,1.0,0.0,1.0,3.0,1.0,...,0.0,0.0,0.0,1.0,1.0,1.0,2.0,1.0,0.0,0
8,80.0,2.0,3347.0,2.0,3.0,5.0,0.0,0.0,3.0,2.0,...,0.0,0.0,0.0,1.0,1.0,1.0,2.0,1.0,0.0,0
10,59.0,1.0,4184.0,1.0,3.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,2.0,2.0,2.0,2.0,3.0,0.0,1
11,78.0,1.0,2413.0,2.0,2.0,2.0,0.0,0.0,3.0,1.0,...,0.0,0.0,0.0,1.0,2.0,2.0,2.0,1.0,0.0,0


In [ ]:
# just checking counts of missing values (not including ones set as 'Missing')
# row with most missing values has 17 missing of 70 columns
nan_counts = df.isna().sum(axis=1)
sorted_nan_counts = nan_counts.sort_values(ascending=False).tolist()

print(sorted_nan_counts)
print(len(sorted_nan_counts))
print(len(df.columns))

[14, 14, 14, 13, 12, 12, 12, 12, 12, 12, 12, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 

In [ ]:
# all columns (except for 6 where we filled in "Missing" for some values) are of type float
# most of our features are categorical with some being float
# we not assign the correct data type to each column
print(df.dtypes.tolist())
df = df.astype('category')
# numerical columns:
# _AGE80, _BMI5, PHYSHLTH, MENTHLTH, POORHLTH, EXEROFT1, EXERHMM1, EXEROFT2,
# EXERHMM2, STRENGTH, CHILDREN, WEIGHT2, HEIGHT3, FALL12MN, FALLINJ5, ALCDAY4,
# AVEDRNK3, DRNK3GE5, MAXDRNKS, DRNKDRI2
numerical_cols = [
    '_AGE80', '_BMI5', 'PHYSHLTH', 'MENTHLTH', 'POORHLTH', 'EXEROFT1',
    'EXERHMM1', 'EXEROFT2', 'EXERHMM2', 'STRENGTH', 'CHILDREN', 'WEIGHT2', 'HEIGHT3', 'FALL12MN', 'FALLINJ5', 'ALCDAY4',
    'AVEDRNK3', 'DRNK3GE5', 'MAXDRNKS', 'DRNKDRI2'
]

df[numerical_cols] = df[numerical_cols].astype(float)

# convert don't know / refused to NaN for numerical columns
# _AGE80 and _BMI5 do not have don't know / refused valued
for col in ['PHYSHLTH', 'MENTHLTH', 'POORHLTH', 'CHILDREN', 'FALL12MN', 'FALLINJ5', 'AVEDRNK3', 'DRNK3GE5', 'MAXDRNKS', 'DRNKDRI2']:
    df[col] = df[col].replace({77: np.nan, 99: np.nan})

for col in ['EXEROFT1', 'EXERHMM1', 'EXEROFT2', 'EXERHMM2', 'STRENGTH', 'ALCDAY4']:
    df[col] = df[col].replace({777: np.nan, 999: np.nan})

for col in ['WEIGHT2', 'HEIGHT3']:
    df[col] = df[col].replace({7777: np.nan, 9999: np.nan})

print(df.dtypes.tolist())

[dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('float64'), dtype('fl

In [ ]:
# double checking that we have the right number of numerical columns
num_cols = df.select_dtypes(include=["float"]).columns.tolist()
print(len(num_cols))
print(len(numerical_cols))

20
20


In [ ]:
df.to_csv('core_processed.csv', index=False)
# note: if loading from csv, data types will have to be re-assigned for numerical and categorical as above

Now that we've processed our core data, we continue processing to construct the initial 3 datasets we will run experiments on.

In [ ]:
# assume MCAR: dropping all missing data
# This reduces our data to 164,751 examples from the initial 197,967
df_drop_missing = df.dropna().copy()
print("Data split with dropped missing values:")
print(df_drop_missing['COVIDSM1'].value_counts(dropna=False))

Data split with dropped missing values:
COVIDSM1
0    142594
1     22157
Name: count, dtype: int64


In [ ]:
# keep missing data, convert don't know / refused for numerical data to missing
# this will be used for classifiers that are able to natively handle missing data
df_keep_missing = df.copy()

print("Data split with keeping missing values:")
print(df_keep_missing['COVIDSM1'].value_counts(dropna=False))

Data split with keeping missing values:
COVIDSM1
0    170893
1     27074
Name: count, dtype: int64


In [ ]:
# assume MAR: keep missing data and impute it
df_impute_missing = df_keep_missing.copy()
categorical_cols = [c for c in df.columns.tolist() if c not in numerical_cols]

In [ ]:
# There are very few missing values in categorical features left (a total of 7 across 5 variables).
# To avoid adding 5 additional features to compensate for only 7 values, and because we just impute "Don't know" for simplicity.
# These could be due to something like human error and the response not being recorded in the survey.
# If it were someone that ended the survey early, they wouldn't have gotten to our target question.
# PRIMINS1    3 missing
df_impute_missing['PRIMINS1'] = df_impute_missing['PRIMINS1'].replace({np.nan:77})
# EDUCA       1 missing
# There actually is no "Don't know" option for education level, so we impute "Refused"
df_impute_missing['EDUCA'] = df_impute_missing['EDUCA'].replace({np.nan:9})
# DECIDE      1 missing
df_impute_missing['DECIDE'] = df_impute_missing['DECIDE'].replace({np.nan:7})
# DIFFALON    1 missing
df_impute_missing['DIFFALON'] = df_impute_missing['DIFFALON'].replace({np.nan:7})
# SEATBELT    1 missing
df_impute_missing['SEATBELT'] = df_impute_missing['SEATBELT'].replace({np.nan:7})

print("Data split with missing values to impute (should be same as keeping missing):")
print(df_keep_missing['COVIDSM1'].value_counts(dropna=False))

Data split with missing values to impute (should be same as keeping missing):
COVIDSM1
0    170893
1     27074
Name: count, dtype: int64


Now we get to actually training and evaluating our classifiers in every case.

In [ ]:
!pip install prince
!pip install lightgbm

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold, cross_validate
from sklearn.experimental import enable_iterative_imputer  # noqa
from sklearn.impute import IterativeImputer

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import (
    RandomForestClassifier,
    AdaBoostClassifier,
    HistGradientBoostingClassifier,
    HistGradientBoostingRegressor
)

from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from prince import FAMD

Helper methods and scikit-learn tranformers for the pipeline

In [ ]:
def compute_scale_pos_weight(y):
    # Compute scale_pos_weight for XGBoost to handle the class imbalance
    # Ratio of the number of negative class examples to positive class examples
    pos = np.sum(y == 1)
    neg = np.sum(y == 0)
    return neg / pos if pos > 0 else 1.0

def get_classifiers(scale_pos_weight):
    # Return a dictionary of classifiers to be used in the pipeline
    # Takes in scale_pos_weight for XGBoost, since for compatability with sklearn pipelines it needs to be precomputed before training
    # Other classifiers account for the class imbalance with the class_weight='balanced' parameter
    return {
        'LogisticRegression': LogisticRegression(random_state=42, max_iter=1000, class_weight='balanced'),
        'DecisionTree': DecisionTreeClassifier(random_state=42, class_weight='balanced'),
        'RandomForest': RandomForestClassifier(random_state=42, class_weight='balanced'),
        'AdaBoost': AdaBoostClassifier(
            random_state=42,
            estimator=DecisionTreeClassifier(max_depth=1, class_weight='balanced')
        ),
        'HistGradientBoosting': HistGradientBoostingClassifier(random_state=42, class_weight='balanced'),
        'XGBoost': XGBClassifier(random_state=42, usse_label_encoder=False, eval_metric='logloss', scale_pos_weight=scale_pos_weight),
        'LightGBM': LGBMClassifier(random_state=42, class_weight='balanced')
    }

def get_classifiers_missing_data(scale_pos_weight):
    # Same as above, but for the experiment where NaN values are left in
    return {
        'DecisionTree': DecisionTreeClassifier(random_state=42, class_weight='balanced'),
        'RandomForest': RandomForestClassifier(random_state=42, class_weight='balanced'),
        'HistGradientBoosting': HistGradientBoostingClassifier(random_state=42, class_weight='balanced'),
        'XGBoost': XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss', scale_pos_weight=scale_pos_weight),
        'LightGBM': LGBMClassifier(random_state=42, class_weight='balanced')
    }

In [ ]:
# remove the target variable from the list of categorical columns for the training pipeline
# since we remove it from X, we do not need to do any transormations or encoding of it
categorical_cols.remove("COVIDSM1")

# if not running from the start, this throws an error

In [ ]:
transformer_no_imputing = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_cols)
    ]
)
transformer_no_imputing.set_output(transform='pandas') # for compatability with FAMD in the pipeline

# categorical columns are handled earlier, so we only do numeric imputation here
transformer_with_imputing = ColumnTransformer([
    ('num', Pipeline([
        ('imputer', IterativeImputer(estimator=HistGradientBoostingRegressor(random_state=42), random_state=42, max_iter=3)),
        ('scaler', StandardScaler())
    ]), numerical_cols)
])
transformer_with_imputing.set_output(transform='pandas')

# we'll perform 5-fold cross validation
# StratifiedKFold preserves the class split in each fold
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

scoring = {
    'accuracy': 'accuracy',
    'precision': 'precision',
    'recall': 'recall',
    'f1': 'f1'
}
metrics = {'accuracy': 'accuracy', 'precision': 'precision', 'recall': 'recall', 'f1': 'f1'}

def evaluate_classifiers(X, y, classifiers, transformer):
    for name, classifier in classifiers.items():
        # without FAMD
        pipeline = Pipeline([
            ('transformer', transformer),
            ('classifier', classifier)
        ])
        pipeline.set_output(transform='pandas')
        scores = cross_validate(pipeline, X, y, cv=cv, scoring=scoring, n_jobs=-1)
        print(f"Results for {name}:")
        print("- Mean Accuracy:", np.mean(scores['test_accuracy']))
        print("- Mean Precision:", np.mean(scores['test_precision']))
        print("- Mean Recall:", np.mean(scores['test_recall']))
        print("- Mean F1:", np.mean(scores['test_f1']))
        print("-" * 50)

        # with FAMD
        # Prince FAMD natively handles categorical vs numeric features in dataframes,
        # so we do not do the column transformations as done before
        pipeline_famd = Pipeline([
            ('famd', FAMD(n_components=200, random_state=42)), # 200 gets ~97% variance explained
            ('classifier', classifier)
        ])
        pipeline_famd.set_output(transform='pandas')
        scores_famd = cross_validate(pipeline_famd, X, y, cv=cv, scoring=scoring, n_jobs=-1)
        print(f"Results for {name} with FAMD:")
        print("- Mean Accuracy:", np.mean(scores_famd['test_accuracy']))
        print("- Mean Precision:", np.mean(scores_famd['test_precision']))
        print("- Mean Recall:", np.mean(scores_famd['test_recall']))
        print("- Mean F1:", np.mean(scores_famd['test_f1']))
        print("-" * 50)

def evaluate_classifiers_no_famd(X, y, classifiers, transformer):
    for name, classifier in classifiers.items():
        # without FAMD
        pipeline = Pipeline([
            ('transformer', transformer),
            ('classifier', classifier)
        ])
        pipeline.set_output(transform='pandas')
        scores = cross_validate(pipeline, X, y, cv=cv, scoring=scoring, n_jobs=-1)
        print(f"Results for {name}:")
        print("- Mean Accuracy:", np.mean(scores['test_accuracy']))
        print("- Mean Precision:", np.mean(scores['test_precision']))
        print("- Mean Recall:", np.mean(scores['test_recall']))
        print("- Mean F1:", np.mean(scores['test_f1']))
        print("-" * 50)

In [ ]:
# df_drop_missing
X_drop_missing = df_drop_missing.drop('COVIDSM1', axis=1)
y_drop_missing = df_drop_missing['COVIDSM1']
spw_drop_missing = compute_scale_pos_weight(y_drop_missing) # for XGBoost

# df_keep_missing
X_keep_missing = df_keep_missing.drop('COVIDSM1', axis=1)
y_keep_missing = df_keep_missing['COVIDSM1']
spw_keep_missing = compute_scale_pos_weight(y_keep_missing)

# df_impute_missing
X_impute_missing = df_impute_missing.drop('COVIDSM1', axis=1)
y_impute_missing = df_impute_missing['COVIDSM1']
spw_impute_missing = compute_scale_pos_weight(y_impute_missing) # again should be same as keep_missing, but just do this for redundancy

Now to run training and evaluation

In [ ]:
print("EXPERIMENT: DROP ALL MISSING VALUES")
classifiers_drop_missing = get_classifiers(spw_drop_missing)
evaluate_classifiers(X_drop_missing, y_drop_missing, classifiers_drop_missing, transformer_no_imputing)

EXPERIMENT: DROP ALL MISSING VALUES
Results for LogisticRegression:
- Mean Accuracy: 0.6895800471468441
- Mean Precision: 0.23815531007066562
- Mean Recall: 0.5949366621937499
- Mean F1: 0.3401442248108698
--------------------------------------------------
Results for LogisticRegression with FAMD:
- Mean Accuracy: 0.6833767146499637
- Mean Precision: 0.23201270590569262
- Mean Recall: 0.5861817727416048
- Mean F1: 0.33241073397241866
--------------------------------------------------
Results for DecisionTree:
- Mean Accuracy: 0.7744657365659379
- Mean Precision: 0.18884319158015295
- Mean Recall: 0.20535280437221512
- Mean F1: 0.196743065701996
--------------------------------------------------
Results for DecisionTree with FAMD:
- Mean Accuracy: 0.7800559508303795
- Mean Precision: 0.18299466925770952
- Mean Recall: 0.18341864668600855
- Mean F1: 0.18319836415477078
--------------------------------------------------
Results for RandomForest:
- Mean Accuracy: 0.8661252464400974
- Mean 

In [ ]:
print("EXPERIMENT: KEEP ALL MISSING VALUES")
classifiers_keep_missing = get_classifiers_missing_data(spw_keep_missing)
evaluate_classifiers_no_famd(X_keep_missing, y_keep_missing, classifiers_keep_missing, transformer_no_imputing)

EXPERIMENT: KEEP ALL MISSING VALUES
Results for DecisionTree:
- Mean Accuracy: 0.7728863802464818
- Mean Precision: 0.19529030121070307
- Mean Recall: 0.2117159268010401
- Mean F1: 0.20316974481644823
--------------------------------------------------
Results for RandomForest:
- Mean Accuracy: 0.8634065249821361
- Mean Precision: 0.5300800163110828
- Mean Recall: 0.010637446570755823
- Mean F1: 0.02085450658281834
--------------------------------------------------
Results for HistGradientBoosting:
- Mean Accuracy: 0.6802850956504864
- Mean Precision: 0.23869713007755716
- Mean Recall: 0.6110662926832762
- Mean F1: 0.34329351110986783
--------------------------------------------------
Results for XGBoost:
- Mean Accuracy: 0.7032131617951862
- Mean Precision: 0.24081057728215663
- Mean Recall: 0.5435843463187162
- Mean F1: 0.3337569784845054
--------------------------------------------------
Results for LightGBM:
- Mean Accuracy: 0.6829825188216914
- Mean Precision: 0.23987800622143213
-

In [ ]:
print("EXPERIMENT: IMPUTE MISSING VALUES")
classifiers_impute_missing = get_classifiers(spw_impute_missing)
evaluate_classifiers(X_drop_missing, y_drop_missing, classifiers_impute_missing, transformer_with_imputing)

EXPERIMENT: IMPUTE MISSING VALUES
Results for LogisticRegression:
- Mean Accuracy: 0.7107210335630102
- Mean Precision: 0.22951886686054893
- Mean Recall: 0.4883335390549191
- Mean F1: 0.3122641860316269
--------------------------------------------------
Results for LogisticRegression with FAMD:
- Mean Accuracy: 0.6833767146499637
- Mean Precision: 0.23201270590569262
- Mean Recall: 0.5861817727416048
- Mean F1: 0.33241073397241866
--------------------------------------------------
Results for DecisionTree:
- Mean Accuracy: 0.7771849840585017
- Mean Precision: 0.17824301879350368
- Mean Recall: 0.18197341180898935
- Mean F1: 0.1800670418450166
--------------------------------------------------
Results for DecisionTree with FAMD:
- Mean Accuracy: 0.7800559508303795
- Mean Precision: 0.18299466925770952
- Mean Recall: 0.18341864668600855
- Mean F1: 0.18319836415477078
--------------------------------------------------
Results for RandomForest:
- Mean Accuracy: 0.8647595469043587
- Mean P

In [ ]:
# we check to see what % of variance explained by each component,
# and the cumulative % variance explained
# this is on the full dataset without a fold removed for cross-validation, so this is just for analysis
famd = FAMD(
    n_components=200,
    n_iter=3,
    copy=True,
    check_input=True,
    random_state=42,
    engine="sklearn",
    handle_unknown="error"
)
famd = famd.fit(X_drop_missing)
famd.eigenvalues_summary

,eigenvalue,% of variance,% of variance (cumulative)
component,,,
0,167.832,3.07%,3.07%
1,128.651,2.36%,5.43%
2,112.001,2.05%,7.48%
3,72.109,1.32%,8.80%
4,62.020,1.14%,9.94%
...,...,...,...
195,14.180,0.26%,95.89%
196,14.136,0.26%,96.15%
197,13.992,0.26%,96.41%
